---
title: "Data Cleaning"
format:
    html: 
        toc: true
        code-fold: false
---

# Code 

Provide the source code used for this section of the project here.

If you're using a package for code organization, you can import it at this point. However, make sure that the **actual workflow steps**—including data processing, analysis, and other key tasks—are conducted and clearly demonstrated on this page. The goal is to show the technical flow of your project, highlighting how the code is executed to achieve your results.

If relevant, link to additional documentation or external references that explain any complex components. This section should give readers a clear view of how the project is implemented from a technical perspective.

Remember, this page is a technical narrative, NOT just a notebook with a collection of code cells, include in-line Prose, to describe what is going on.

# U.S. Exoneration Data Cleaning

*Data Cleaning Process*
- Removed columns with more than 50% missing values to focus on more complete data.
- Standardized column names and formats for easier manipulation.
- Addressed missing values in key columns:
  - Filled missing `county` values with "Unknown."
  - Dropped rows with missing `age` values as they are critical for analysis.
- Cleaned `tags` column by removing extraneous symbols (`#`, `;`) and ensuring uniform formatting.


In [28]:
# Import necessary Libraries
import pandas as pd 
import numpy as np 
import re

# Load exoneration dataset
df = pd.read_csv('../../data/raw-data/US_exoneration_data.csv')
print("Initial Dataset: ")
df.head()


Initial Dataset: 


,Last Name,First Name,Age,Race,Sex,State,County,Tags,Worst Crime Display,Sentence,...,F/MFE,FC,ILD,P/FA,DNA,MWID,OM,Date of Exoneration,Date of 1st Conviction,Date of Release
0,Abbitt,Joseph,31.0,Black,Male,North Carolina,Forsyth,CV;#IO;#SA,Child Sex Abuse,Life,...,NaN,NaN,NaN,NaN,DNA,MWID,NaN,9/2/09,6/22/95,9/2/09
1,Abbott,Cinque,19.0,Black,Male,Illinois,Cook,CIU;#IO;#NC;#P,Drug Possession or Sale,Probation,...,NaN,NaN,NaN,P/FA,NaN,NaN,OM,2/1/22,3/25/08,3/25/08
2,Abdal,Warith Habib,43.0,Black,Male,New York,Erie,IO;#SA,Sexual Assault,20 to Life,...,F/MFE,NaN,NaN,NaN,DNA,MWID,OM,9/1/99,6/6/83,9/1/99
3,Abernathy,Christopher,17.0,White,Male,Illinois,Cook,CIU;#CV;#H;#IO;#JV;#SA,Murder,Life without parole,...,NaN,FC,NaN,P/FA,DNA,NaN,OM,2/11/15,1/15/87,2/11/15
4,Abney,Quentin,32.0,Black,Male,New York,New York,CV,Robbery,20 to Life,...,NaN,NaN,NaN,NaN,NaN,MWID,NaN,1/19/12,3/20/06,1/19/12


In [29]:
# Managing Missing Data - Identifying which columns have a lot of missing data
na_counts = df.isna().sum()
print(na_counts)

Last Name                    0
First Name                   0
Age                         27
Race                         0
Sex                          0
State                        0
County                      66
Tags                       171
Worst Crime Display          0
Sentence                     0
Posting Date                 0
OM Tags                   1430
F/MFE                     2557
FC                        3133
ILD                       2602
P/FA                      1291
DNA                       2984
MWID                      2610
OM                        1430
Date of Exoneration          0
Date of 1st Conviction       0
Date of Release              0
dtype: int64


In [30]:
# Drop columns with excessive missing values 
df.drop(columns = ['OM Tags', 'F/MFE', 'ILD', 'P/FA', 'DNA', 'MWID', 'FC','OM'], inplace = True)

# Standardize column names by converting to lowercase and replacing spaces with '_'
df.columns = df.columns.str.lower().str.replace(' ', '_')

# Convert sex values to lowercase 
df['sex'] = df['sex'].str.lower()

print(df.columns)


Index(['last_name', 'first_name', 'age', 'race', 'sex', 'state', 'county',
       'tags', 'worst_crime_display', 'sentence', 'posting_date',
       'date_of_exoneration', 'date_of_1st_conviction', 'date_of_release'],
      dtype='object')


*Illinois Subset:*
- Filtered the dataset to focus on cases from Illinois, resulting in a subset of 548 rows to be used for further analysis.


In [31]:
# Filter Data for Illinois 
df = df[df['state'] == 'Illinois']
print("Number of rows: " , IL_exonerations.shape[0]) 
df.head()

Number of rows:  548


,last_name,first_name,age,race,sex,state,county,tags,worst_crime_display,sentence,posting_date,date_of_exoneration,date_of_1st_conviction,date_of_release
1,Abbott,Cinque,19.0,Black,male,Illinois,Cook,CIU;#IO;#NC;#P,Drug Possession or Sale,Probation,2/14/22,2/1/22,3/25/08,3/25/08
3,Abernathy,Christopher,17.0,White,male,Illinois,Cook,CIU;#CV;#H;#IO;#JV;#SA,Murder,Life without parole,2/13/15,2/11/15,1/15/87,2/11/15
5,Abrego,Eruby,20.0,Hispanic,male,Illinois,Cook,CDC;#H;#IO,Murder,90 years,8/25/22,7/21/22,9/22/04,7/21/22
10,Adams,Demetris,22.0,Black,male,Illinois,Cook,CIU;#IO;#NC;#P,Drug Possession or Sale,1 year,4/13/20,2/11/20,9/8/04,12/26/04
15,Adams,Kenneth,22.0,Black,male,Illinois,Cook,CDC;#H;#IO;#JI;#SA,Murder,75 years,8/29/11,7/2/96,10/20/78,6/14/96


break tags into individual columns for later eda

In [32]:
# Clean tags column by removing unecesary '#' from tags column
df['tags'] = df['tags'].str.replace('#', '', regex=False).str.replace(";", ",")


# Define the mapping for tags
tag_mapping = {
    "A": "arson",
    "BM": "bitemark",
    "CDC": "co_defendant_confessed",
    "CIU": "conviction_integrity_unit",
    "CSH": "child_sex_abuse_hysteria_case",
    "CV": "child_victim",
    "F": "female_exoneree",
    "FED": "federal_case",
    "H": "homicide",
    "IO": "innocence_organization",
    "JI": "jailhouse_informant",
    "JV": "juvenile_defendant",
    "M": "misdemeanor",
    "NC": "no_crime_case",
    "P": "guilty_plea_case",
    "PH": "posthumous_exoneration",
    "SA": "sexual_assault",
    "SBS": "shaken_baby_syndrome_case",
    "PR": "prosecutor_misconduct",
    "OF": "police_officer_misconduct",
    "FA": "forensic_analyst_misconduct",
    "CW": "child_welfare_worker_misconduct",
    "WH": "withheld_exculpatory_evidence",
    "NW": "misconduct_that_is_not_withholding_evidence",
    "KP": "knowingly_permitting_perjury",
    "WT": "witness_tampering_or_misconduct_interrogating_co_defendant",
    "INT": "misconduct_in_interrogation_of_exoneree",
    "PJ": "perjury_by_official",
    "PL": "prosecutor_lied_in_court"
}

# Ensure the 'tags' column is a list of tags (split if needed)
df['tags'] = df['tags'].apply(lambda x: x.split(',') if isinstance(x, str) else x)

# Create a binary column for each tag
for tag in tag_mapping.keys():
    df[tag] = df['tags'].apply(lambda x: 1 if isinstance(x, list) and tag in x else 0)

# Rename the binary columns using the tag_mapping dictionary
df.rename(columns=tag_mapping, inplace=True)

# Create `tag_sum` column to count the total number of tags for each exoneree
df['tag_sum'] = df[list(tag_mapping.values())].sum(axis=1)

#Drop tag columns
df.drop(columns=['tags'], inplace=True)

df.head()

,last_name,first_name,age,race,sex,state,county,worst_crime_display,sentence,posting_date,...,forensic_analyst_misconduct,child_welfare_worker_misconduct,withheld_exculpatory_evidence,misconduct_that_is_not_withholding_evidence,knowingly_permitting_perjury,witness_tampering_or_misconduct_interrogating_co_defendant,misconduct_in_interrogation_of_exoneree,perjury_by_official,prosecutor_lied_in_court,tag_sum
1,Abbott,Cinque,19.0,Black,male,Illinois,Cook,Drug Possession or Sale,Probation,2/14/22,...,0,0,0,0,0,0,0,0,0,4
3,Abernathy,Christopher,17.0,White,male,Illinois,Cook,Murder,Life without parole,2/13/15,...,0,0,0,0,0,0,0,0,0,6
5,Abrego,Eruby,20.0,Hispanic,male,Illinois,Cook,Murder,90 years,8/25/22,...,0,0,0,0,0,0,0,0,0,3
10,Adams,Demetris,22.0,Black,male,Illinois,Cook,Drug Possession or Sale,1 year,4/13/20,...,0,0,0,0,0,0,0,0,0,4
15,Adams,Kenneth,22.0,Black,male,Illinois,Cook,Murder,75 years,8/29/11,...,0,0,0,0,0,0,0,0,0,5


In [33]:
# Print unique sentencing values for a better idea on how to best clean column 
unique_sentences = df['sentence'].unique()
print(unique_sentences)

['Probation' 'Life without parole' '90 years' '1 year' '75 years'
 '30 years' '55 years' '2 years' '3 years' '6 years' '45 years'
 '1 year and 6 months' '50 years' '60 years' 'Life' '80 years' '18 years'
 '4 years' '85 years' '20 years' '35 years' '2 years and 6 months'
 '82 years' '12 years' 'Not sentenced' '22 years' '32 years' 'Death'
 '5 years' '40 years' '25 years' '26 years' '4 years and 6 months'
 '9 years' '48 years' '30 days' '84 years' '3 months and 25 days'
 '2 years and 2 months' '3 months' '44 years' '6 months' '25 to 50 years'
 '29 years' '23 years' '31 years' '11 years' '8 years' '24 years'
 '3 years and four months' '42 years' '3 years and 6 months' '65 years'
 '76 years' '15 years' '50 to Life' '86 years' '70 years' '28 years'
 '13 years' '47 years' '36 years' '18 months' '1 year and 4 months'
 '8 years and 6 months' '6 years and 6 months' '58 years' '95 years'
 '7 years' '34 years' '62 years' '27 years' '69 years' '57 years'
 '50 to 100 years' '4 months' '4 years and 

In [39]:
def clean_sentence(value):
    """ Cleans the 'sentence' column values to numeric years for numerical EDA 
    - Probation is represented as 0.
    - 'Not sentenced' is converted to np.nan.
    - 'Life' and 'Death' sentences are represented as 100 (placeholder).
    - Years and months are converted to a numeric value in years. """

    if value == 'Probation':
        return 0
    elif value == 'Not sentenced':
        return np.nan  # NaN for not sentenced
    elif 'Life' in value or value == 'Death':
        return 100  # Placeholder for life sentences or death penalty
    elif 'year' in value or 'month' in value:

        # Handles ranges like '25 to 50 years'
        if 'to' in value:
            years = [int(num) for num in re.findall(r'\d+', value)]
            return sum(years) / len(years)  # Average the range 
        
        # Handle "X years and Y months"
        elif 'and' in value:
            numbers = [float(num) for num in re.findall(r'\d+', value)]
            if len(numbers) == 2:  # Both years and months are present
                years, months = numbers
                return years + (months / 12)  # Convert months to years
            elif len(numbers) == 1:  # Only one number is present
                return numbers[0]  # Treat it as years
            
        # Handle only months or only years
        elif 'months' in value:
            months = int(re.search(r'\d+', value).group())
            return months / 12  # Convert months to years
        else:  # Only years
            return int(re.search(r'\d+', value).group())
    else:
        return np.nan  # Anything unexpected as None
    
df['sentence_in_years'] = df['sentence'].apply(clean_sentence)

# Check results
df[['sentence', 'sentence_in_years']].head(10)

,sentence,sentence_in_years
1,Probation,0.0
3,Life without parole,100.0
5,90 years,90.0
10,1 year,1.0
15,75 years,75.0
21,Probation,0.0
22,Probation,0.0
24,30 years,30.0
25,55 years,55.0
45,1 year,1.0


In [ ]:
# Convert date columns into datetime format
for col in ['posting_date', 'date_of_exoneration', 'date_of_1st_conviction', 'date_of_release']:
    try:
        # Attempt to convert with explicit format (MM/DD/YY)
        df[col] = pd.to_datetime(df[col], format='%m/%d/%y', errors='coerce')
    except Exception as e:
        print(f"Error processing column {col}: {e}")
        df[col] = pd.to_datetime(df[col], errors='coerce')  # Fallback to auto-detection

print(df[['posting_date', 'date_of_exoneration', 'date_of_1st_conviction', 'date_of_release']].head())

   posting_date date_of_exoneration date_of_1st_conviction date_of_release
1    2022-02-14          2022-02-01             2008-03-25      2008-03-25
3    2015-02-13          2015-02-11             1987-01-15      2015-02-11
5    2022-08-25          2022-07-21             2004-09-22      2022-07-21
10   2020-04-13          2020-02-11             2004-09-08      2004-12-26
15   2011-08-29          1996-07-02             1978-10-20      1996-06-14


In [43]:
# Reordering columns to place 'sentence_in_years' after 'sentence'
columns = list(df.columns)  # Get all columns
columns.insert(columns.index('sentence') + 1, columns.pop(columns.index('sentence_in_years')))  # Move 'sentence_in_years'
df = df[columns]  # Reorder DataFrame

# Display the updated DataFrame
df.head()
pd.set_option('display.max_columns', None)
df.head(10)

,last_name,first_name,age,race,sex,state,county,worst_crime_display,sentence,sentence_in_years,posting_date,date_of_exoneration,date_of_1st_conviction,date_of_release,arson,bitemark,co_defendant_confessed,conviction_integrity_unit,child_sex_abuse_hysteria_case,child_victim,female_exoneree,federal_case,homicide,innocence_organization,jailhouse_informant,juvenile_defendant,misdemeanor,no_crime_case,guilty_plea_case,posthumous_exoneration,sexual_assault,shaken_baby_syndrome_case,prosecutor_misconduct,police_officer_misconduct,forensic_analyst_misconduct,child_welfare_worker_misconduct,withheld_exculpatory_evidence,misconduct_that_is_not_withholding_evidence,knowingly_permitting_perjury,witness_tampering_or_misconduct_interrogating_co_defendant,misconduct_in_interrogation_of_exoneree,perjury_by_official,prosecutor_lied_in_court,tag_sum
1,Abbott,Cinque,19.0,Black,male,Illinois,Cook,Drug Possession or Sale,Probation,0.0,2022-02-14,2022-02-01,2008-03-25,2008-03-25,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
3,Abernathy,Christopher,17.0,White,male,Illinois,Cook,Murder,Life without parole,100.0,2015-02-13,2015-02-11,1987-01-15,2015-02-11,0,0,0,1,0,1,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6
5,Abrego,Eruby,20.0,Hispanic,male,Illinois,Cook,Murder,90 years,90.0,2022-08-25,2022-07-21,2004-09-22,2022-07-21,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
10,Adams,Demetris,22.0,Black,male,Illinois,Cook,Drug Possession or Sale,1 year,1.0,2020-04-13,2020-02-11,2004-09-08,2004-12-26,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4
15,Adams,Kenneth,22.0,Black,male,Illinois,Cook,Murder,75 years,75.0,2011-08-29,1996-07-02,1978-10-20,1996-06-14,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,5
21,Adams,Seneca,20.0,Black,male,Illinois,Cook,Assault,Probation,0.0,2014-12-08,2006-12-19,2006-05-18,2006-05-18,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
22,Adams,Tari,18.0,Black,male,Illinois,Cook,Assault,Probation,0.0,2014-12-08,2006-12-19,2006-05-18,2006-05-18,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
24,Agnew,Gregory,28.0,Black,male,Illinois,Lake,Robbery,30 years,30.0,2018-01-18,2001-11-07,1988-06-14,2001-11-07,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
25,Aguirre,Omar,28.0,Hispanic,male,Illinois,Cook,Murder,55 years,55.0,2011-08-29,2003-02-18,1999-03-09,2002-12-18,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
45,Ali,Chauncey,37.0,Black,male,Illinois,Cook,Drug Possession or Sale,1 year,1.0,2020-04-13,2020-02-11,2007-01-17,2007-06-27,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4


In [44]:
df.to_csv('../../data/processed-data/illinois_exoneration_data.csv', index=False)

print("Data saved to 'illinois_exoneration_data.csv'")

Data saved to 'illinois_exoneration_data.csv'
